## Generación de base de datos de palabras en Español

In [1]:
# librerias utilizadas
import pandas as pd
import nltk
import string
import datetime
from collections import Counter
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /home/jogabell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jogabell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Lectura de base de datos
gh_2015=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2015',encode='latin1')
gh_2016=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2016',encode='latin1')
gh_2017=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2017',encode='latin1')
gh_2018=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2018',encode='latin1')
gh_2019=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2019',encode='latin1') 

In [3]:
#gh_2015.head

In [4]:
features=[u'Project ID', u'Name', u'Organization',u'Published Status',
       u'Goal', u'Percentage Raised', u'Number of Individual Donors',  
       u'Short DescSP', u'Long Desc SP']
_2015=gh_2015[features]
_2016=gh_2016[features]
_2017=gh_2017[features]
_2018=gh_2018[features]
_2019=gh_2019[features]

In [5]:
_2015['Published Status'].value_counts()

ended       175
draft        87
private       3
rejected      1
Name: Published Status, dtype: int64

In [6]:
# Limpiado base de datos. Se quitando registros con 'Published Status'='draft' o 'rejected'

data=_2015.append(_2016).append(_2017).append(_2018).append(_2019)

In [7]:
data.shape

(3966, 9)

In [8]:
mask1=(data['Published Status']!= 'draft')

In [9]:
data['Published Status'].value_counts()

draft        3105
ended         749
rejected       70
publish        28
private        10
submitted       2
archived        1
Name: Published Status, dtype: int64

In [10]:
data=data[mask1]

In [11]:
data['Published Status'].value_counts()

ended        749
rejected      70
publish       28
private       10
submitted      2
archived       1
Name: Published Status, dtype: int64

In [12]:
mask2=(data['Published Status']!= 'rejected')

In [13]:
data=data[mask2]

In [14]:
data['Published Status'].value_counts()

ended        749
publish       28
private       10
submitted      2
archived       1
Name: Published Status, dtype: int64

## Creación de base de datos de texto para proyectos que no recaudaron

In [15]:
mask3=data[u'Percentage Raised']<=50

In [16]:
data_cero_donation=data[mask3]

In [17]:
mask4=data[u'Number of Individual Donors']<=0

In [18]:
data_cero_donation=data[mask4]

In [19]:
data_cero_donation.shape

(150, 9)

## Creación de base de datos de texto para proyectos que alcanzaron o rebasaron la meta

In [20]:
mask5=data['Percentage Raised']>=100

In [21]:
data_goal_donation=data[mask5]

In [22]:
mask6=data_goal_donation['Number of Individual Donors']>=1

In [23]:
data_goal_donation=data_goal_donation[mask6]

In [24]:
data_goal_donation.shape

(134, 9)

## Función para preparar los datos de texto

In [45]:
def text_analysis(serie):
    s=serie.isna().value_counts()    
    if s.size==2:
          serie=serie.dropna()
    elif (s.size==1)&(s[0]==True):
      print('no existen datos')
    datetime_object = datetime.datetime.now()
    lista=[]    
    for u_text in serie:
        if(type(u_text)!= type(datetime_object)):            
            lista.append(u_text.encode("utf-8"))
    text_string_type=', '.join(lista)
    text_string_type=unicode(text_string_type, "utf-8").lower()    
    
    useless_words=nltk.corpus.stopwords.words("spanish")+nltk.corpus.stopwords.words("english")+list(string.punctuation)+[u'cada',u'través',u'para',u'\xa1que',u'\xa1to',u'son',u'con',u'la',u'se',u'en',u'el',u'que',u'la',u'las',u'lo',u'los',u'de',u'the',u'us',u'we',u'A',u'nbsp',u'8217',u'8211',u'8220',u'8221',u'\u2019',u'\u201d',u'\u201c',u'pa\xeds',u'visi\xf3n',u'ray\xf3n',u'pa\xeds',u'tecnolog\xeda',u'ay\xfadanos',u'\xa1que',u'harder\u2014but',u'michoac\xe1n',u'let\xb4s',u'\xa1to',u'granny\xb4s',u'espa\xf1ol.',u'.']#+list(u"\u2019")+list('...')    
    words=nltk.word_tokenize(text_string_type)
    words_filtered=[word for word in words if not word in useless_words]
    
    words_counter=Counter(words_filtered)
    most_common_words = words_counter.most_common()
    return most_common_words

## Obtención de short desciption en Español

In [46]:
# Proyectos con recaudación cero y recaudación igual o superior al objetivo
short_text_cero_donations=data_cero_donation['Short DescSP']
short_text_goal_donations=data_goal_donation['Short DescSP']

In [47]:
short_text_cero_donations=short_text_cero_donations.dropna()
short_text_goal_donations=short_text_goal_donations.dropna()

In [48]:
short_text_cero_donations.shape

(132,)

In [49]:
short_text_goal_donations.shape

(131,)

In [50]:
#short_text_cero_donations y short_text_goal_donations

In [51]:
mcw_cero_donations=text_analysis(short_text_cero_donations)
mcw_goal_donations=text_analysis(short_text_goal_donations)

In [52]:
#def transform_words(serie):
#    result=[]
#    for s in serie:
#        #s[0]=s[0].encode('utf-8')
#        print(s[0])
#        result.append((s[0].encode('utf-8'),s[1]))
#        #string.replace(s[0].decode('latin1'), u'\xf1', 'ñ')
#    return result

In [53]:
#mcw_cero_donations

In [54]:
#mcw_goal_donations

In [55]:
## Creación de base de datos para realizar gráficos de la short description en Español
df_mcw_goal_donations=pd.DataFrame(mcw_goal_donations)
df_mcw_goal_donations.to_excel('data/short_description_words_goal_ESP.xlsx')

In [56]:
df_mcw_cero_donations=pd.DataFrame(mcw_cero_donations)
df_mcw_cero_donations.to_excel('data/short_description_words_cero_ESP.xlsx')

## Obtención de long description en Español

In [57]:
# Proyectos con recaudación cero y recaudación igual o superior al objetivo
long_text_cero_donations=data_cero_donation['Long Desc SP']
long_text_goal_donations=data_goal_donation['Long Desc SP']

In [58]:
long_text_cero_donations=long_text_cero_donations.dropna()
long_text_goal_donations=long_text_goal_donations.dropna()

In [59]:
# Palabras en Español más utilizadas en las descripciones largas 
# para proyectos que recaudaron el objetivo o más y para los que 
# no recaudaron
mcw_long_cero_donations=text_analysis(long_text_cero_donations)
mcw_long_goal_donations=text_analysis(long_text_goal_donations)

In [60]:
#mcw_long_cero_donations

In [61]:
#mcw_long_goal_donations

In [62]:
# Guardando base de datos de las palabras más utilizadas en las descripciones largas en Español
df_mcw_long_cero_donations=pd.DataFrame(mcw_long_cero_donations)
df_mcw_long_goal_donations=pd.DataFrame(mcw_long_goal_donations)

df_mcw_long_cero_donations.to_excel('data/long_description_words_cero_ESP.xlsx')
df_mcw_long_goal_donations.to_excel('data/long_description_words_goal_ESP.xlsx')


In [63]:
# Se ha terminado el proceso y limpieza de los datos de texto en Espalñol